# Neural Networks (ANN) – Step-by-Step (Binary Classification)
Train a simple **Neural Network** to predict **Pass / Fail** using two features:
- Hours studied
- Practice tests taken

This notebook covers:
1) Data setup
2) Train/test split
3) Scaling
4) Build a small network (Dense layers)
5) Train + evaluate
6) Predict for a new student


In [ ]:
# If TensorFlow is missing in your environment, uncomment the next line:
# !pip -q install tensorflow

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

print('TensorFlow version:', tf.__version__)

## 1) Create a small dataset

In [ ]:
X = np.array([
    [1, 0],
    [2, 0],
    [2, 1],
    [3, 1],
    [3, 2],
    [4, 2],
    [5, 2],
    [6, 3],
    [7, 3],
    [8, 4],
])
y = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1])  # 0=Fail, 1=Pass

print('X shape:', X.shape)
print('y shape:', y.shape)
print('First 5 rows of X:\n', X[:5])
print('First 5 labels:', y[:5])

## 2) Train/Test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
print('Train size:', len(X_train))
print('Test size:', len(X_test))

## 3) Scale features

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Before scaling (first train row):', X_train[0])
print('After scaling  (first train row):', X_train_scaled[0])

## 4) Build a small Neural Network

In [ ]:
model = Sequential([
    Dense(8, activation='relu', input_shape=(2,)),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

## 5) Train the model (with EarlyStopping)

In [ ]:
early_stop = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)

history = model.fit(
    X_train_scaled, y_train,
    epochs=300,
    verbose=0,
    callbacks=[early_stop]
)

print('Training finished. Epochs used:', len(history.history['loss']))

## 6) Evaluate

In [ ]:
loss, acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', acc)

## 7) Predict a new student

In [ ]:
new_student = np.array([[4, 1]])
new_student_scaled = scaler.transform(new_student)

prob = float(model.predict(new_student_scaled, verbose=0)[0][0])
pred = int(prob >= 0.5)

print('New student [hours, practice_tests]:', new_student[0])
print('Probability of Pass:', prob)
print('Prediction (0=Fail, 1=Pass):', pred)

## 8) Optional: plot training loss

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(history.history['loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()